<a href="https://colab.research.google.com/github/AmiraBelkis/tsp-ant-colony/blob/AmiraBelkis-Param-1/Tabu_static.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabu method

In [2]:
"""
#importing from github
!git clone https://github.com/AmiraBelkis/tsp-ant-colony.git
import sys
sys.path.append('/content/tsp-ant-colony')
"""
# importing neccessairy packages
import numpy as np
import random
import argparse
import Parser
import time
import copy


def generate_first_solution_PPV(graphe, v_depart=None):
     # Faire une copie du graphe vu qu'il va subir des modification
    _graphe = graphe.copy()
    # La liste chemin garde trace de notre parcour
    chemin = []
    # Selection d'1 point de depart
    if v_depart is None:
        depart = v_depart = np.random.randint(0, len(graphe))
    depart = v_depart
    # Ajouter le point de depart au chamin de parcour
    chemin.append(v_depart)
    # Création de l'ensemble des noeuds non visités
    noeudsNonVisite = set(
        np.delete(np.arange(0, len(graphe)), v_depart).flatten())
    cout = 0
    while (len(noeudsNonVisite) != 0):
        # Retourne le plus proche voisin
        v_suivante = np.argmin(_graphe[v_depart, :])
        # màj du chemin
        chemin.append(v_suivante)
        # màj du cout
        cout += _graphe[v_depart, v_suivante]
        # Aller au prochain neoud
        noeudsNonVisite.remove(v_suivante)
        v_depart = v_suivante
        # De/vers les noeuds deja visité a l'infini
        _graphe[v_depart, chemin] = float("inf")
        _graphe[chemin, v_depart] = float("inf")

    # Ajouter le cout de retour
    cout += graphe[v_suivante, depart]

    return chemin, cout


def trouver_voisinage(solution, matrice):
    neighborhood_of_solution = []

    for n in solution[1:-1]:
        idx1 = solution.index(n)
        for kn in solution[idx1+1 :-1]:
            idx2 = solution.index(kn)
            # Calcul du voisin
            _tmp = copy.deepcopy(solution)
            _tmp[idx1] = kn
            _tmp[idx2] = n

            distance = 0
            for i in range(len(matrice)):
                #Calculer le cout du voisin 
                distance += matrice[_tmp[i - 1]][_tmp[i]]

            _tmp.append(distance)

            #Ajouter le resultat à la liste des voisins 
            neighborhood_of_solution.append(_tmp)

    indexOfLastItemInTheList = len(neighborhood_of_solution[0]) - 1
    # Trier les voisins suivant leurs couts 
    neighborhood_of_solution.sort(key=lambda x: x[indexOfLastItemInTheList])
    return neighborhood_of_solution


def recherche_tabou(matrice, iters, size, start_node=None):
    # Generation de la solution initiale
    solution, best_cost = generate_first_solution_PPV(matrice, start_node)
    # Initialisation de la liste tabou
    tabu_list = list() 
    best_solution_ever = solution
    # Repeter pour un nombre d'iterations predefinie
    for count in range(iters): 
        # Generation des voisins de la solution
        neighborhood = trouver_voisinage(solution, matrice)
        # commencer par le voisin avec le cout le plus minimal
        index_of_best_solution = 0
        best_solution = neighborhood[index_of_best_solution]
        best_cost_index = len(best_solution) - 1

        found = False
        while found is False:
            
            if best_solution not in tabu_list:
                #si la solution n'existe pas a la liste tabou
                tabu_list.append(best_solution[1: -2])
                found = True
                solution = best_solution[:-1]
                cost = neighborhood[index_of_best_solution][best_cost_index]
                if cost < best_cost:
                    best_cost = cost
                    best_solution_ever = solution
            else:
                index_of_best_solution = index_of_best_solution + 1
                best_solution = neighborhood[index_of_best_solution]

        if len(tabu_list) >= size:
            _ = tabu_list.pop(0)

    return best_solution_ever, best_cost




# Generating a data set


In [ ]:
import time
import random
# writing data result to a CSV file to use it in ANOVA 
import csv 
    
# field names                                                    
fields = ['data_set_size','iterations', 'size', 'start', 'cost', 'time']
# data rows of csv file 
rows = [] 
    
# name of csv file 
filename = "/content/data_tabu1000.csv"    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)       
    # writing the fields 
    csvwriter.writerow(fields) 
    datasetList = ['/content/tsp-ant-colony/data/data_101.tsp']
    for dataset in datasetList: 
        instance = Parser.TSPInstance(dataset)
        instance.readData()
        data = np.copy(instance.data)
        # numbre of nodes in data set     
        ln = len(data[0])
        # Tabu loop
        for i1 in range(1000):
            # numbre of iterations
            itr = random.randint(1,101)
            # size of tabu list 
            size = random.randint(1,51)
            # starting node 
            start = random.randint(0,ln-1)
            start_time = time.time()
            tour, cost = recherche_tabou(data, itr,size, start)
            end_time = time.time()
            row =[ln,itr ,size,start, 1000/cost,  0.01/(end_time-start_time)]
            rows.append(row)
            csvwriter.writerow(row)
print(rows[0:10])

In [5]:
ln = len(data[0])
print(ln)

101
